In [9]:
import requests
import json
import pandas as pd
import pycountry
from datetime import datetime, date
from sqlalchemy import create_engine
from datetime import datetime
import cx_Oracle   # pip install cx_Oracle

# Replace with your Oracle database connection details
USERNAME = "RM94134"
PASSWORD = "181089"
HOST = "oracle.fiap.com.br"
PORT = "1521"
SID = "ORCL"
TARGET_TABLE = 'FBI_INTERPOL_WANTED_CRIMINALS'




In [2]:
# Install Oracle Client https://www.oracle.com/database/technologies/instant-client/downloads.html
# Copy the address where the instant client was unzipped
lib_dir = r"C:\Users\asale\OneDrive\Documentos\libdir"

# ------------------------------ UTILITIES --------------------------------
def remove_keys_from_dict(the_dict: dict, keys: list):
    for key in keys:
        if key in the_dict:
            del the_dict[key]

def format_date(date_str):
    try:
        date_obj = pd.to_datetime(date_str)
        return date_obj.strftime('%B %d, %Y')
    except ValueError:
        return str(date_str)  # Return the original value as string if it's not a valid date    

def list_to_string(the_list):
    try:
        joined_string = '; '.join(the_list)
        return joined_string
    except:
        return the_list         

def walk_json_path(json_obj, *args):
    inner_value = json_obj
    for arg in args:
        try:
            current_value = inner_value.get(arg, '')
            if current_value == {}:
                break
            inner_value = inner_value.get(arg, '')
        except:
            return ''
    return inner_value           

def add_html_paragraph_tags(string):
    strings_with_tags = '<p>' + string + '</p>'
    return strings_with_tags

def get_country_name(country_id):  # 2 letters country code
    try:
        country_obj = pycountry.countries.get(alpha_2=country_id)
        if country_obj != None: 
            return country_obj.name
        return country_id
    except:
        return country_id     

def transform_float_feet_height_to_cm_string(h):
    try:
        feet = int(h / 10)
        inches = int(h % 10)

        feet_cm = feet * 30.48
        inches_cm = inches * 2.54
        
        meters = float(feet_cm + inches_cm) / 100
        
        meters_str = f"{meters:.2f}"
        return meters_str
    except:
        return h   

def get_age_from_date_of_birth(date_string):
    date_format = "%B %d, %Y"
    try:
        date_then = datetime.strptime(date_string, date_format).date()
        current_date = date.today()
        age_in_years = str(int((current_date - date_then).days / 365.25)) + ' years old'  # Account for leap years
        return age_in_years
    except:
        return ''           

def SQL_CREATE_STATEMENT_FROM_DATAFRAME(source_df, target_table_name):
    only_text_data_type_df = source_df.astype(str)
    sql_text = pd.io.sql.get_schema(only_text_data_type_df, target_table_name)   
    oracle_clean_statement = sql_text.replace('\"', '').replace('TEXT', 'CLOB')
    return oracle_clean_statement

def SQL_INSERT_STATEMENT_FROM_DATAFRAME(source_df, target_table_name):
    sql_texts = []
    for index, row in source_df.iterrows():
        columns_str = ', '.join(source_df.columns)
        values_list = [str(value).replace("'", "''") if value is not None else 'NULL' for value in row.values]
        values_str = ', '.join([f"'{value}'" if value != 'NULL' else 'NULL' for value in values_list])
        insert_statement = f"INSERT INTO {target_table_name} ({columns_str}) VALUES ({values_str})"
        sql_texts.append(insert_statement)
    return sql_texts        

In [3]:
       
# --------------------------- CALL FBI API --------------------------------
''
# Create an empty DataFrame
fbi_wanted_df = pd.DataFrame()

page = 1
while True:
    response = requests.get('https://api.fbi.gov/wanted/v1/list', params={'page': page})
    data = json.loads(response.content)

    if data['total'] == 0 or data['items'] == []:
        print('No data left to fetch')
        break

    response_items = data['items']

    # Ensure all keys are present in every dictionary
    all_keys = list(set().union(*(item.keys() for item in response_items)))
    for item in response_items:
        for k in all_keys:
            item.setdefault(k, None)

    # Create a DataFrame
    df = pd.DataFrame(response_items, columns=all_keys)

    # Concatenate the DataFrame to the existing DataFrame
    fbi_wanted_df = pd.concat([fbi_wanted_df, df], axis=0, sort=True, ignore_index=True)
    
    print(f'FBI search page is: {page}')
    page += 1
    if page == 199:     # Loop safety check
        print('Something\'s wrong, page iteration is at 199')
        break           
print(f'Number of rows fetched from FBI is: {len(fbi_wanted_df)}')
print('Success! Finished pulling data from FBI API')

FBI search page is: 1
FBI search page is: 2
FBI search page is: 3
FBI search page is: 4
FBI search page is: 5
FBI search page is: 6
FBI search page is: 7
FBI search page is: 8
FBI search page is: 9
FBI search page is: 10
FBI search page is: 11
FBI search page is: 12
FBI search page is: 13
FBI search page is: 14
FBI search page is: 15
FBI search page is: 16
FBI search page is: 17
FBI search page is: 18
FBI search page is: 19
FBI search page is: 20
FBI search page is: 21
FBI search page is: 22
FBI search page is: 23
FBI search page is: 24
FBI search page is: 25
FBI search page is: 26
FBI search page is: 27
FBI search page is: 28
FBI search page is: 29
FBI search page is: 30
FBI search page is: 31
FBI search page is: 32
FBI search page is: 33
FBI search page is: 34
FBI search page is: 35
FBI search page is: 36
FBI search page is: 37
FBI search page is: 38
FBI search page is: 39
FBI search page is: 40
FBI search page is: 41
FBI search page is: 42
FBI search page is: 43
FBI search page is: 

In [4]:

# --------------------------- CALL INTERPOL API --------------------------------

page = 1
resultPerPage = 160
#notices_iterated = 0

interpol_wanted_df = pd.DataFrame()
while True:
    response = requests.get('https://ws-public.interpol.int/notices/v1/red', params={'page': page, 'resultPerPage': resultPerPage})
    try:
        interpol_data = json.loads(response.content)
    except:
        break
    page += 1

    # Get all red notices
    red_notices = interpol_data['_embedded']['notices']

    # Get each red notice inner content
    for notice in red_notices:
        try:
            more_details_url = notice['_links']['self']['href']
            more_details_json = json.loads(requests.get(more_details_url, timeout=5).content) 

            # Handle images
            thumbnail_url = walk_json_path(notice, '_links', 'thumbnail', 'href') 
            images_url = walk_json_path(notice, '_links', 'images', 'href')
            try:
                larger_image_url = json.loads(requests.get(images_url).content)['_embedded']['images'][0]['_links']['self']['href']
                images_dict = {'thumb': thumbnail_url, 'large': larger_image_url}  # Set dict to match FBI response keys 
            except:
                images_dict = {}

            # Remove useless keys
            keys_to_remove_from_dict = ['_links', 'thumbnail', '_embedded']
            remove_keys_from_dict(notice, keys_to_remove_from_dict)
            remove_keys_from_dict(more_details_json, keys_to_remove_from_dict)

            full_notices_dict = {**notice, **more_details_json}  # Unpacking dicts into one
            full_notices_dict['images'] = str(images_dict)

            df =  pd.DataFrame([full_notices_dict])

            # Explode JSON columns
            arrest_warrants = pd.concat([pd.json_normalize(record) for record in df['arrest_warrants']], ignore_index=True)
            df = pd.concat([df.drop(columns='arrest_warrants'), arrest_warrants], axis=1)

            interpol_wanted_df = pd.concat([interpol_wanted_df, df], axis=0, sort=True, ignore_index=True)     
        except Exception as e:
            print('An exception happened. Skipping notice: ') 
            print(notice)
            print(e)
            continue

    print(f'Interpol search page is: {page}. Number of rows fetched from Interpol is: {len(interpol_wanted_df)}')
    if page == 99:   # Loop safety check
        print('Something\'s wrong, page index is 99 in code')
        break      
print('Success! Finished pulling data from INTERPOL API')


Interpol search page is: 2. Number of rows fetched from Interpol is: 166
Interpol search page is: 3. Number of rows fetched from Interpol is: 332
Interpol search page is: 4. Number of rows fetched from Interpol is: 498
Interpol search page is: 5. Number of rows fetched from Interpol is: 664
Interpol search page is: 6. Number of rows fetched from Interpol is: 830
Interpol search page is: 7. Number of rows fetched from Interpol is: 996
Interpol search page is: 8. Number of rows fetched from Interpol is: 1162
Interpol search page is: 9. Number of rows fetched from Interpol is: 1328
Interpol search page is: 10. Number of rows fetched from Interpol is: 1494
Interpol search page is: 11. Number of rows fetched from Interpol is: 1660
Interpol search page is: 12. Number of rows fetched from Interpol is: 1826
Interpol search page is: 13. Number of rows fetched from Interpol is: 1992
Interpol search page is: 14. Number of rows fetched from Interpol is: 2158
Interpol search page is: 15. Number of 

Interpol search page is: 55. Number of rows fetched from Interpol is: 8958
Interpol search page is: 56. Number of rows fetched from Interpol is: 9124
Interpol search page is: 57. Number of rows fetched from Interpol is: 9290
Interpol search page is: 58. Number of rows fetched from Interpol is: 9456
Interpol search page is: 59. Number of rows fetched from Interpol is: 9622
Interpol search page is: 60. Number of rows fetched from Interpol is: 9788
Interpol search page is: 61. Number of rows fetched from Interpol is: 9954
Interpol search page is: 62. Number of rows fetched from Interpol is: 10120
Interpol search page is: 63. Number of rows fetched from Interpol is: 10286
Interpol search page is: 64. Number of rows fetched from Interpol is: 10452
Interpol search page is: 65. Number of rows fetched from Interpol is: 10618
Interpol search page is: 66. Number of rows fetched from Interpol is: 10784
Interpol search page is: 67. Number of rows fetched from Interpol is: 10950
Interpol search pag

In [5]:
 

# -------------------- Transforming and uniting data from both sources ------------------------
interpol_wanted_df['wanted_origin'] = 'INTERPOL'
fbi_wanted_df['wanted_origin'] = 'FBI'    

fbi_wanted_df.rename(
    columns={
        'scars_and_marks':'distinguishing_marks',
        'caution':'charges',
        'eyes':'eyes_color',
        'hair':'hair_color'
    }, 
    inplace=True
)

interpol_wanted_df.rename(
    columns={
        'date_of_birth':'dates_of_birth_used',
        'charge': 'charges',
        'sex_id': 'sex',
        'country_of_birth_id': 'nationality',
        'eyes_colors_id':'eyes_color',
        'hairs_id':'hair_color',
        'languages_spoken_ids': 'languages'
    }, 
    inplace=True
)

interpol_wanted_df['dates_of_birth_used'] = pd.to_datetime(interpol_wanted_df['dates_of_birth_used'])
interpol_wanted_df['dates_of_birth_used'] = interpol_wanted_df['dates_of_birth_used'].apply(format_date)
interpol_wanted_df['age_range'] = interpol_wanted_df['dates_of_birth_used'].apply(get_age_from_date_of_birth)
fbi_wanted_df['dates_of_birth_used'] = fbi_wanted_df['dates_of_birth_used'].apply(list_to_string)

interpol_wanted_df['charges'] = interpol_wanted_df['charges'].apply(add_html_paragraph_tags)

fbi_wanted_df['aliases'] = fbi_wanted_df['aliases'].apply(list_to_string)
interpol_wanted_df['aliases'] = (interpol_wanted_df['forename'] + ' ' + interpol_wanted_df['name']).str.title()
interpol_wanted_df['forename'] = interpol_wanted_df['forename'].str.title()
interpol_wanted_df['name'] = interpol_wanted_df['name'].str.title()

sex_mapping = {'M': 'Male', 'F': 'Female'}
interpol_wanted_df['sex'] = interpol_wanted_df['sex'].map(sex_mapping)

interpol_wanted_df['eyes_color'] = interpol_wanted_df['eyes_color'].apply(list_to_string)
interpol_wanted_df['hair_color'] = interpol_wanted_df['hair_color'].apply(list_to_string)

fbi_wanted_df['height_max'] = fbi_wanted_df['height_max'].apply(transform_float_feet_height_to_cm_string)
fbi_wanted_df['height_min'] = fbi_wanted_df['height_min'].apply(transform_float_feet_height_to_cm_string)
fbi_wanted_df['height'] = fbi_wanted_df['height_min'] + ';' + fbi_wanted_df['height_max']

interpol_wanted_df['languages'] = interpol_wanted_df['languages'].apply(list_to_string)
fbi_wanted_df['languages'] = fbi_wanted_df['languages'].apply(list_to_string)

interpol_wanted_df['nationality'] = interpol_wanted_df['nationality'].apply(get_country_name)
interpol_wanted_df['issuing_country_id'] = interpol_wanted_df['issuing_country_id'].apply(get_country_name)
fbi_wanted_df['issuing_country_id'] = 'United States of America'

interpol_wanted_df['wanted_origin_id'] = interpol_wanted_df['entity_id']
fbi_wanted_df['wanted_origin_id'] = fbi_wanted_df['uid']

fbi_wanted_df['field_offices'] = fbi_wanted_df['field_offices'].apply(list_to_string)
fbi_wanted_df['occupations'] = fbi_wanted_df['occupations'].apply(list_to_string)
fbi_wanted_df['possible_countries'] = fbi_wanted_df['possible_countries'].apply(list_to_string)
fbi_wanted_df['possible_states'] = fbi_wanted_df['possible_states'].apply(list_to_string)
fbi_wanted_df['subjects'] = fbi_wanted_df['subjects'].apply(list_to_string)
interpol_wanted_df['images'] = interpol_wanted_df['images'].apply(lambda item: [item])

interpol_wanted_df.drop('entity_id', axis=1, inplace=True)
interpol_wanted_df.drop('charge_translation', axis=1, inplace=True)
interpol_wanted_df.drop('nationalities', axis=1, inplace=True)
fbi_wanted_df.drop('path', axis=1, inplace=True)
fbi_wanted_df.drop('legat_names', axis=1, inplace=True)
fbi_wanted_df.drop('locations', axis=1, inplace=True)
fbi_wanted_df.drop('files', axis=1, inplace=True)
fbi_wanted_df.drop('coordinates', axis=1, inplace=True)
fbi_wanted_df.drop('uid', axis=1, inplace=True)
fbi_wanted_df.drop('@id', axis=1, inplace=True)
fbi_wanted_df.drop('additional_information', axis=1, inplace=True)
fbi_wanted_df.drop('description', axis=1, inplace=True)
fbi_wanted_df.drop('reward_max', axis=1, inplace=True)
fbi_wanted_df.drop('reward_min', axis=1, inplace=True)

merged_df = pd.concat([interpol_wanted_df, fbi_wanted_df], axis=0, ignore_index=True)
merged_df['weight'].replace(0, None, inplace=True)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
merged_df['analyzed_at'] = timestamp
print('Success! Finished transforming and uniting data into a single dataframe.')

Success! Finished transforming and uniting data into a single dataframe.


In [18]:


# -------------------------------------- SAVE TO ORACLE DATABASE --------------------------------------
try:
    cx_Oracle.init_oracle_client(lib_dir=lib_dir)
except:  # may have been initialized already
    pass

dsn = cx_Oracle.makedsn(HOST, PORT, sid=SID)
connection = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn)
cursor = connection.cursor()

try:  # Check if table exists
    pd.read_sql(f'SELECT * FROM {TARGET_TABLE}', connection)
except:
    try:
        cursor.execute(SQL_CREATE_STATEMENT_FROM_DATAFRAME(merged_df, TARGET_TABLE))
        print('Table created')
    except Exception as e:
        print('Table not created')
        print(e)


C:\Users\asale\AppData\Local\Temp\ipykernel_27700\877455970.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(f'SELECT * FROM {TARGET_TABLE}', connection)


Table created
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
ORA-01704: string literal too long
Success! Script is finished.


In [19]:
# Write the DataFrame to Oracle database 
insert_statements = SQL_INSERT_STATEMENT_FROM_DATAFRAME(merged_df, TARGET_TABLE)
for statement in insert_statements:
    try:
        cursor.execute(statement)
    except Exception as e:
        print(e)

connection.commit()  # Commit the changes
cursor.close()  # Close the cursor

print('Success! Script is finished.')


not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
not open
n

InterfaceError: not open